# 1. ETL(Extraccion, Transformación y Carga)

## 1.Extracción (Extract)

In [ ]:
from google.colab import drive
import pandas as pd
import chardet

# Montamos Google Drive para acceder al archivo
drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/results.csv' or '/content/results.csv'
with open(ruta_archivo, 'rb') as f:
  result = chardet.detect(f.read())
  print (result)
try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=',', decimal='.')
  print('Extración de datos exitosa')
except FileNotFoundError:
  print(f'El archivo no se encuentra en la ruta: {ruta_archivo}')
except Exception as e:
  print(f'Ocurrió un error al leer el archivo: {e}')

Mounted at /content/drive
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
Extración de datos exitosa


## 2.Transformacion(Transform)

In [ ]:
 # Muestra las primeras 10 filas del DataFrame
df.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,False
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,False


In [ ]:
# Resumen de la información del DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48673 entries, 0 to 48672
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        48673 non-null  object
 1   home_team   48673 non-null  object
 2   away_team   48673 non-null  object
 3   home_score  48673 non-null  int64 
 4   away_score  48673 non-null  int64 
 5   tournament  48673 non-null  object
 6   city        48673 non-null  object
 7   country     48673 non-null  object
 8   neutral     48673 non-null  bool  
dtypes: bool(1), int64(2), object(6)
memory usage: 3.0+ MB


In [ ]:
# Verifica tipos, en este caso es object
df.dtypes

,0
date,object
home_team,object
away_team,object
home_score,int64
away_score,int64
tournament,object
city,object
country,object
neutral,bool


In [ ]:
# Estadisticas descriptivas de las columnas numéricas
df.describe()

,home_score,away_score
count,48673.000000,48673.000000
mean,1.757874,1.181743
std,1.772685,1.401651
min,0.000000,0.000000
25%,1.000000,0.000000
50%,1.000000,1.000000
75%,2.000000,2.000000
max,31.000000,21.000000


In [ ]:
# Eliminamos filas con valores faltantes críticos
df.dropna(subset=['home_team', 'away_team', 'home_score', 'away_score'], inplace=True)

# Eliminamos duplicados
df.drop_duplicates(inplace=True)
print("Transformación de datos exitosa")

Transformación de datos exitosa


In [ ]:
# Transforamar date en datatime
df['date'] = pd.to_datetime(df['date'])


print("Transformación de datos exitosa")

Transformación de datos exitosa


In [ ]:
# Renombrar columnas para facilitar el acceso y evitar caracteres especiales
df.rename(columns ={
    'date': 'Date',
    'home_team': 'Home Team',
    'away_team': 'Away Team',
    'home_score': 'Home Score',
    'away_score': 'Away Score',
    'tournament': 'Tournament',
    'city':'City',
    'country': 'Country',
    'neutral': 'Neutral'
}, inplace=True)

In [ ]:
# Diferencia de goles
df['DifGoles'] = df['Home Score'] - df['Away Score']
print(df)

            Date            Home Team Away Team  Home Score  Away Score  \
0     1872-11-30             Scotland   England           0           0   
1     1873-03-08              England  Scotland           4           2   
2     1874-03-07             Scotland   England           2           1   
3     1875-03-06              England  Scotland           2           2   
4     1876-03-04             Scotland   England           3           0   
...          ...                  ...       ...         ...         ...   
48668 2025-10-14  Republic of Ireland   Armenia           1           0   
48669 2025-10-14              Estonia   Moldova           1           1   
48670 2025-10-14                Italy    Israel           3           0   
48671 2025-10-14              Andorra    Serbia           1           3   
48672 2025-10-14               Latvia   England           0           5   

                         Tournament     City              Country  Neutral  \
0                    

In [ ]:
# Resultado del Partido (ganó, empató, perdió)
def ResultadosPartidos (row):
    if row['Home Score'] > row['Away Score']:
        return ' Ganó Local'
    elif row['Home Score'] < row['Away Score']:
        return 'Ganó Visitante'
    else:
        return 'Empate'

df['Resultado'] = df.apply(ResultadosPartidos, axis=1)


In [ ]:
df.head()

,Date,Home Team,Away Team,Home Score,Away Score,Tournament,City,Country,Neutral,DifGoles,Resultado
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False,0,Empate
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False,2,Ganó Local
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False,1,Ganó Local
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False,0,Empate
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False,3,Ganó Local


## 3.Carga (Load)

In [ ]:
# definir la ruta de salida para guardar el nuevo CSV
ruta_salida = '/content/drive/MyDrive/ResultadosFulbo.csv'

# Guardar el dataframe transformado en un nuevo archivo CSV en Drive
df.to_csv(ruta_salida, index=False, sep=',')

print(f'Archivo CSV guardado en: {ruta_salida}')

Archivo CSV guardado en: /content/drive/MyDrive/ResultadosFulbo.csv


# 2. - EDA (Análisis Exploratorio de datos)


In [ ]:
from google.colab import drive
import pandas as pd
import chardet

# Montamos Google Drive para acceder al archivo
drive.mount('/content/drive', force_remount=True)

ruta_archivo = '/content/drive/MyDrive/ResultadosFulbo.csv'
with open(ruta_archivo, 'rb') as f:
  result = chardet.detect(f.read())
  print (result)
try:
  df = pd.read_csv(ruta_archivo, encoding=result['encoding'], sep=',', decimal='.')
  print('Extración de datos exitosa')
except FileNotFoundError:
  print(f'El archivo no se encuentra en la ruta: {ruta_archivo}')
except Exception as e:
  print(f'Ocurrió un error al leer el archivo: {e}')

Mounted at /content/drive
{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}
Extración de datos exitosa


In [ ]:
# Información general del nuevo dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48673 entries, 0 to 48672
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Date        48673 non-null  object
 1   Home Team   48673 non-null  object
 2   Away Team   48673 non-null  object
 3   Home Score  48673 non-null  int64 
 4   Away Score  48673 non-null  int64 
 5   Tournament  48673 non-null  object
 6   City        48673 non-null  object
 7   Country     48673 non-null  object
 8   Neutral     48673 non-null  bool  
 9   DifGoles    48673 non-null  int64 
 10  Resultado   48673 non-null  object
dtypes: bool(1), int64(3), object(7)
memory usage: 3.8+ MB


In [ ]:
# Estadisticas descriptivas de las columnas numéricas del nuevo DataFrame
df.describe()

,Home Score,Away Score,DifGoles
count,48673.000000,48673.000000,48673.000000
mean,1.757874,1.181743,0.576131
std,1.772685,1.401651,2.413784
min,0.000000,0.000000,-21.000000
25%,1.000000,0.000000,-1.000000
50%,1.000000,1.000000,0.000000
75%,2.000000,2.000000,2.000000
max,31.000000,21.000000,31.000000


In [ ]:
# ¿Podemos ver en el dataset los partidos de Argentina? Filtramos los partidos donde Argentina fue local o visitante
PartidosArgentina = df[
    (df['Home Team'] == 'Argentina') | (df['Away Team'] == 'Argentina')
]

# Mostramos las primeras filas
PartidosArgentina.head()

,Date,Home Team,Away Team,Home Score,Away Score,Tournament,City,Country,Neutral,DifGoles,Resultado
145,1902-07-20,Uruguay,Argentina,0,6,Friendly,Montevideo,Uruguay,False,-6,Ganó Visitante
155,1903-09-13,Argentina,Uruguay,2,3,Friendly,Buenos Aires,Argentina,False,-1,Ganó Visitante
180,1905-08-15,Argentina,Uruguay,0,0,Copa Lipton,Buenos Aires,Argentina,False,0,Empate
193,1906-08-15,Uruguay,Argentina,0,2,Copa Lipton,Montevideo,Uruguay,False,-2,Ganó Visitante
195,1906-10-21,Argentina,Uruguay,2,1,Copa Newton,Buenos Aires,Argentina,False,1,Ganó Local


In [ ]:
# ¿ Cuantos partidos jugó Argentina ? Imprimimos el total de partidos Jugados de Argentina
print("Total de partidos jugados por Argentina:", len(PartidosArgentina))


Total de partidos jugados por Argentina: 1061


In [ ]:
# ¿ Como salieron esos partidos? Determinamos el resultado desde el punto de vista de Argentina
def ResultadoArgentina(row):
    if row['Home Team'] == 'Argentina' and row['Home Score'] > row['Away Score']:
        return 'Victoria'
    elif row['Away Team'] == 'Argentina' and row['Away Score'] > row['Home Score']:
        return 'Victoria'
    elif row['Home Score'] == row['Away Score']:
        return 'Empate'
    else:
        return 'Derrota'

PartidosArgentina['ResultadoArgentina'] = PartidosArgentina.apply(ResultadoArgentina, axis=1)

# Conteo
PartidosArgentina['ResultadoArgentina'].value_counts(normalize=False) * 1

/tmp/ipython-input-4197115589.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  PartidosArgentina['ResultadoArgentina'] = PartidosArgentina.apply(ResultadoArgentina, axis=1)


,count
ResultadoArgentina,
Victoria,585
Empate,257
Derrota,219


In [ ]:
# Creamos una tabla pivote donde apreciamos los partidos por fecha con total al final
tabla = pd.pivot_table(
    data=PartidosArgentina,
    index='Date',
    values=['Home Score', 'Away Score'],
    aggfunc='sum'
)

tabla.loc['Total'] = tabla.sum()
print(tabla)


            Away Score  Home Score
Date                              
1902-07-20           6           0
1903-09-13           3           2
1905-08-15           0           0
1906-08-15           2           0
1906-10-21           1           2
...                ...         ...
2025-09-04           0           3
2025-09-09           0           1
2025-10-10           0           1
2025-10-14           6           0
Total             1172        1907

[1054 rows x 2 columns]
